# Databaze

Databaze sluze za pohranu podataka u nekom formatu, te jednostavan pristup

MySQL i ostale SQL baze koriste relacijsko povezane podatke

Ovdje mozes pogledati malo vise o konceptu Entity Relationship vezama, idealno bi bilo da procitas sve od ER Diagrams pa nadolje (20ak stranica), ali ove su neke bitne TL;DR sekcije


https://www.relationaldbdesign.com/database-design/module1/intro-relational-database-design.php
 - [ER Diagrams](https://www.relationaldbdesign.com/database-design/module6/intro-entityRelationship-diagrams.php)
 - [Asocijacije entiteta](https://www.relationaldbdesign.com/database-design/module6/entity-relationship-concept.php)
 - [Tipovi veza](https://www.relationaldbdesign.com/database-design/module6/three-relationship-types.php)
 - [One to One](https://www.relationaldbdesign.com/database-design/module6/identify-oneToOne-relationships.php)
 - [One to Many](https://www.relationaldbdesign.com/database-design/module6/one-to-manyRelationships.php)

Podatci mogu biti povezani na razne nacine, to su:
 - one to one - 1:1 (Jedna osoba ima tocno jedan OIB, jedan OIB ima tocno jednu osobu)
 - one to many - 1:N (Jedna osoba može imati puno auta, auto pripada samo jednoj osobi)
 - many to many - N:N (Jedan autobus ima puno putnika, jedan putnik vozi puno autobusa)
 - self referencing (Jedna osoba ima jednog voditelja (osobu), jedan voditelj ima više članova tima (osobe))

Note: self referencing moze bit one to one ili one to many ovisno o tome kako definiramo tu vezu. Primjer naveden gore je one to many.


Programski jezici koriste objektno orijentirane podatke, te im pristupamo kroz atribute

## Primjer objektno orijentiranih podataka

Imamo objekt knjiznica s vezom one to many na objekt knjige

Dodajemo nove knjige u knjiznicu

Pristupamo knjigama koje pripadaju nekoj knjiznici


In [61]:
# Definicija Knjiznice
class Library:
    def __init__(self, id, name, location):
        self.id = id
        self.name = name
        self.location = location
        self.books = []

# Definicija Knjige        
class Book:
    def __init__(self, id, name, year):
        self.id = id
        self.name = name
        self.year = year
    def __repr__(self): # pomocna funkcija koja nam pomogne u printanju objekta 
        return "("+str(self.id)+", "+self.name+", "+str(self.year)+")"

# Kreiramo novi podatak - knjiznica
library = Library(1, "Alexandria", "Egypt")

# Dodajemo knjige kao pripadnike knjiznice
library.books.append( Book(1, "Harry Potter", 1997) )
library.books.append( Book(2, "The Hobbit", 1937) )

# Pristupamo podatcima
print(library.books)

[(1, Harry Potter, 1997), (2, The Hobbit, 1937)]


# Primjer relacijsko povezanih podataka

Ukoliko zelimo povezati nesto u relacijskoj bazi podataka to nemozemo vezati direktno na objekt
Vec trebamo kreirati vanjski kljuc na objektu koji ce referencirati vezu

Pravila su sljedeca: 
 - Svaka tablica mora imati stupac ID
 - To znaci da svaki redak u tablici ima unikatnu ID vrijednost (primary key)
 - Imamo foreign key u tablici knjiga 
 - Ako u foreign key zapisemo postojeci ID iz tablice knjiznica imamo vezu izmedu ta dva objekta

Tako u ovom slucaju imamo sljedecu schemu
```
+--------------------+           +-------------------+
|        KNJIGA      |           |    KNJIZNICA      |
|====================|           |===================|
|  ID        |int    |   +------>|  ID      | int    |
| name       |varchar|   |       | name     | varchar|
| year       |int    |   |       | location | varchar|
| library_id |int    |---+       +-------------------+
+--------------------+           
```

In [62]:
# SQLite je 'portabl' SQL, malo je opusteniji no uglavnom je ista funkcionalnost i koncepti su primjenjivi na oboje
# https://www.geeksforgeeks.org/differences-between-sql-and-sqlite/
import sqlite3
conn = sqlite3.connect(':memory:') 

# Definicija Knjiznice
conn.execute("CREATE TABLE Library(id, name, location)")

# Definicija Knjige - postavljamo vanjski kljuc koji referencira Knjiznicu
conn.execute("CREATE TABLE Book(id, name, year, library_id)")

# Kreiramo novi podatak - knjiznica
conn.execute("INSERT INTO Library(id, name, location) VALUES(1, 'Alexandria', 'Egypt')")

# Dodajemo knjige kao pripadnike knjiznice
conn.execute("INSERT INTO Book(id, name, year, library_id) VALUES(1, 'Harry Potter', 1997, 1)")
conn.execute("INSERT INTO Book(id, name, year, library_id) VALUES(2, 'The Hobbit'  , 1937, 1)")

# Pristupamo podatcima
print(conn.execute("SELECT * FROM Book WHERE library_id = 1").fetchall())

[(1, 'Harry Potter', 1997, 1), (2, 'The Hobbit', 1937, 1)]


### Pogledaj si ovu odlicnu prezu s FERa o relacijsko vs objektno orijentiranim podatcima
https://www.fer.unizg.hr/_download/repository/3._OOBP_i_ORBP.pdf

## Foreign keys
Foreign key constraint je dobro meta pravilo 
 - jer tako se baza pripremi na tu vezu 
 - ubrza pristupanje tim podatcima
 - pri kreiranju podataka validira da li je veza moguca 

Bitno je znati da veze mogu postojati i bez ovog meta pravila (constraint-a)

https://www.cockroachlabs.com/blog/what-is-a-foreign-key/

https://www.w3schools.com/sql/sql_foreignkey.asp

In [63]:
import sqlite3
# https://www.tutorialspoint.com/sqlite/sqlite_python.htm#:~:text=To%20use%20sqlite3%20module%2C%20you,executing%20all%20the%20SQL%20statements.

conn = sqlite3.connect(':memory:')
print("Opened database successfully");

# https://www.w3schools.com/sql/sql_syntax.asp
conn.execute("""CREATE TABLE COMPANY
         (ID INT PRIMARY KEY     NOT NULL,
         NAME           TEXT    NOT NULL,
         AGE            INT     NOT NULL,
         ADDRESS        CHAR(50),
         SALARY         REAL);""")


conn.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (1, 'Paul', 32, 'California', 20000.00 )");

conn.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (2, 'Allen', 25, 'Texas', 15000.00 )");

conn.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (3, 'Teddy', 23, 'Norway', 20000.00 )");

conn.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (4, 'Mark', 25, 'Rich-Mond ', 65000.00 )");

conn.commit()


cursor = conn.execute("SELECT id, name, address, salary from COMPANY")
for row in cursor:
   print("ID = ", row[0])
   print("NAME = ", row[1])
    
conn.close()


Opened database successfully
ID =  1
NAME =  Paul
ID =  2
NAME =  Allen
ID =  3
NAME =  Teddy
ID =  4
NAME =  Mark


In [64]:
# https://www.apachefriends.org/
# MariaDB & phpMyAdmin
